# Getting started

## Installation 

[`frds`](/) requires Python3.8 or higher.

### Install via `PyPI`

Using `pip` is the simplest way. To install using `pip`:

```bash
pip install frds --upgrade
```

### Install from source

[`frds`](https://github.com/mgao6767/frds/) is available on GitHub. To download the source code and install:

```bash
cd ~
git clone https://github.com/mgao6767/frds.git
cd frds
pip install .
```



## Setup

By default, a folder named `frds` will be created under the user's home directory to store downloaded data.

`frds` primarily uses WRDS to obtain data, so it requires WRDS login credentials. The setup is done by `frds.io.wrds.setup`:

In [1]:
from frds.io.wrds import setup

If `save_credentials=True`, the username and password will be saved locally in `credentials.json` in the `frds` folder. Then in later uses, no more setup is required (no just current session).

In [2]:
setup(username='username', password='password', save_credentials=False)

## Usage

### Compute metrics on the go
A typical example of how to use `frds` in a few lines.

In [3]:
import pandas as pd
from frds.data.wrds.comp import funda
from frds.io.wrds import load

FUNDA = load(funda, use_cache=True, obs=100)

(FUNDA.PPENT / FUNDA.AT).to_frame("Tangibility")

Tangibility
GVKEY  DATADATE               
001000 1961-12-31          NaN
       1962-12-31          NaN
       1963-12-31          NaN
       1964-12-31     0.397599
       1965-12-31     0.604762
...                        ...
001004 1982-05-31     0.286231
       1983-05-31     0.287057
       1984-05-31     0.245198
       1985-05-31     0.221782
       1986-05-31     0.194738

[100 rows x 1 columns]

### Buil-in measures

The real time-saver is the built-in measures in `frds.measures`.

In [4]:
from frds.measures import ROA

(roa_v1 := ROA(FUNDA).to_frame("ROA: IB/AT"))

ROA: IB/AT
GVKEY  DATADATE              
001000 1961-12-31         NaN
       1962-12-31         NaN
       1963-12-31         NaN
       1964-12-31    0.027542
       1965-12-31   -0.085281
...                       ...
001004 1982-05-31    0.010778
       1983-05-31    0.025115
       1984-05-31    0.032697
       1985-05-31    0.058370
       1986-05-31    0.058027

[100 rows x 1 columns]

Some measures may have variants. For example, ROA can be either computed as the income before extraordinary items scaled by contemporaneous total assets, or by lagged total assets which can be calculated by `ROA.v2`.

In [5]:
(roa_v2 := ROA.v2(FUNDA).to_frame("ROA: IB/lagged AT"))

ROA: IB/lagged AT
GVKEY  DATADATE                     
001000 1961-12-31                NaN
       1962-12-31                NaN
       1963-12-31                NaN
       1964-12-31                NaN
       1965-12-31          -0.139124
...                              ...
001004 1982-05-31           0.014746
       1983-05-31           0.024592
       1984-05-31           0.040319
       1985-05-31           0.066102
       1986-05-31           0.074039

[100 rows x 1 columns]

We could then perform some easy analyses on the computed metrics.

In [6]:
# Pearson correlation
roa_v1.join(roa_v2).corr()

,ROA: IB/AT,ROA: IB/lagged AT
ROA: IB/AT,1.000000,0.786803
ROA: IB/lagged AT,0.786803,1.000000
